# World Bank - Mortality Summarizing
Reduced '.csv' files from WB_data Notebook

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import requests
import matplotlib.pyplot as plt
from scipy.stats import linregress
import numpy as np
import time


# Impor the OpenWeatherMap API key
from api_keys import geoapify_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

In [2]:
# WB data set
WB_africadata_2000_bins = Path("Resources\WB_africadata_2000_bins.csv")

In [3]:
# Read data file with the Pandas library
# encoding?, i.e encoding="ISO-8859-1"
WB_africadata_2000_df2 = pd.read_csv(WB_africadata_2000_bins)

In [4]:
WB_africadata_2000_df2

,series_id,country_code,country_name,year,value,Year Group
0,SH.HIV.INCD.TL,KEN,Kenya,2017,3.400000e+04,2016 - 2020
1,SH.HIV.INCD.TL,LBR,Liberia,2017,1.600000e+03,2016 - 2020
2,SH.HIV.INCD.TL,LSO,Lesotho,2017,1.000000e+04,2016 - 2020
3,SH.HIV.INCD.TL,MDG,Madagascar,2017,7.200000e+03,2016 - 2020
4,SH.HIV.INCD.TL,MOZ,Mozambique,2017,1.300000e+05,2016 - 2020
...,...,...,...,...,...,...
652157,VA.NO.SRC,WSM,Samoa,2006,3.000000e+00,2006 - 2010
652158,FM.LBL.BMNY.GD.ZS,TCD,Chad,2010,1.146678e+01,2006 - 2010
652159,NE.CON.PRVT.CN.AD,SDN,Sudan,2000,2.199464e+10,2000 - 2005
652160,NE.CON.PRVT.CN.AD,SDN,Sudan,2003,3.190503e+10,2000 - 2005


# Slicing of the Data
###In order to analyze the data and to understand the impact of the Foreign Aid with regards to poverty rate, literacy and mortality, the data was sliced by the "series_id". The deifinition of the diferent "series_id" values (indicators) is described in a second file downloaded from the Nasdaq API and saved as 'WB_metadata_df'.

###To facilitate/expedite the search and filtering of the relevant a function was created to search the text strings : "key_word". This function is case sensitive and it's used is combined with the review of the search output selection to identify the indicators that best match our work objective.

In [5]:
# WB id series definition data set
WB_metadata = Path("Resources/WB_METADATA_f7ce7fba293ccc6eb39cdf15fb097982.csv")

In [6]:
# Read data file with the Pandas library
WB_metadata_df = pd.read_csv(WB_metadata)

In [7]:
WB_metadata_df

,series_id,name,description
0,DC.DAC.DEUL.CD,"Net bilateral aid flows from DAC donors, Germa...",Net bilateral aid flows from DAC donors are th...
1,RQ.STD.ERR,Regulatory Quality: Standard Error,Regulatory Quality captures perceptions of the...
2,EG.USE.PCAP.KG.OE,Energy use (kg of oil equivalent per capita),Energy use refers to use of primary energy bef...
3,EN.POP.EL5M.UR.ZS,Urban population living in areas where elevati...,Urban population below 5m is the percentage of...
4,per_lm_alllm.cov_q1_tot,Coverage of unemployment benefits and ALMP in ...,Coverage of unemployment benefits and active l...
...,...,...,...
1479,SL.TLF.BASC.FE.ZS,"Labor force with basic education, female (% of...",The ratio of the labor force with basic educat...
1480,SL.GDP.PCAP.EM.KD,GDP per person employed (constant 2021 PPP $),GDP per person employed is gross domestic prod...
1481,SL.TLF.INTM.MA.ZS,"Labor force with intermediate education, male ...",The ratio of the labor force with intermediate...
1482,SL.UEM.NEET.FE.ZS,"Share of youth not in education, employment or...","Share of youth not in education, employment or..."


In [8]:
# This function will search within a text string for a given 'key word'. It is case sensitive

def key_word(df, search_col, word_txt, result_col):
    i=0   
    result_ls = []
    for item in df[search_col]:
        text_ls = [x.strip() for x in df[search_col][i].split()]
        if word_txt in text_ls:
            result_ls.append(df[result_col][i])
        i+=1
    if i >= df[search_col].count():
        if len(result_ls) > 0:
            return result_ls
        else:
            return print("Keyword Not found")

In [9]:
Mortality_search = key_word(WB_metadata_df, 'name', 'Mortality', 'series_id')

In [10]:
len(Mortality_search)

20

In [11]:
Mortality_search2= WB_metadata_df[WB_metadata_df['series_id'].isin(Mortality_search)]

In [12]:
Mortality_search2

,series_id,name,description
35,SH.STA.AIRP.MA.P5,Mortality rate attributed to household and amb...,Mortality rate attributed to household and amb...
100,SH.DYN.NCOM.FE.ZS,"Mortality from CVD, cancer, diabetes or CRD be...","Mortality from CVD, cancer, diabetes or CRD is..."
106,SP.DYN.AMRT.FE,"Mortality rate, adult, female (per 1,000 femal...","Adult mortality rate, female, is the probabili..."
186,SH.DYN.MORT.FE,"Mortality rate, under-5, female (per 1,000 liv...","Under-five mortality rate, female is the proba..."
239,SH.DYN.MORT.MA,"Mortality rate, under-5, male (per 1,000 live ...","Under-five mortality rate, male is the probabi..."
281,SH.STA.WASH.P5,"Mortality rate attributed to unsafe water, uns...","Mortality rate attributed to unsafe water, uns..."
297,SP.DYN.IMRT.MA.IN,"Mortality rate, infant, male (per 1,000 live b...","Infant mortality rate, male is the number of m..."
354,SP.DYN.IMRT.IN,"Mortality rate, infant (per 1,000 live births)",Infant mortality rate is the number of infants...
356,SH.DYN.MORT,"Mortality rate, under-5 (per 1,000 live births)",Under-five mortality rate is the probability p...
481,SP.DYN.AMRT.MA,"Mortality rate, adult, male (per 1,000 male ad...","Adult mortality rate, male, is the probability..."


# Selection of Indicators for Analysis

###Indicators are selected out of the search result and used to create redu ed data sets. In the case of the Mortality indicator, we selected three indicators : mortality under age 5-females only, mortality under age 5-males only, and mortality under age 5-combined. Values listed are the rate per 100,000 parts.

###In order to organize the data, we prepared 6 data sets. One for each indicator.

In [13]:
# Thought these indicators might provide more data for the Mortality analysis: rate is per 100000 females, males, population
#SH.DYN.MORT.FE - mortality under age 5, females
#SH.DYN.MORT.MA - mortality under age 5, males
#SH.DYN.MORT - mortality under age 5, combined

In [14]:
WB_Mortality_df = WB_africadata_2000_df2.loc[WB_africadata_2000_df2['series_id'].isin(["SH.DYN.MORT.FE", "SH.DYN.MORT.MA", "SH.DYN.MORT" ])]
WB_Mortality_df

,series_id,country_code,country_name,year,value,Year Group
127992,SH.DYN.MORT.FE,MDG,Madagascar,2016,60.1,2016 - 2020
196033,SH.DYN.MORT.FE,TUN,Tunisia,2018,14.1,2016 - 2020
301671,SH.DYN.MORT.FE,AGO,Angola,2000,190.9,2000 - 2005
301672,SH.DYN.MORT.FE,BFA,Burkina Faso,2000,171.5,2000 - 2005
301673,SH.DYN.MORT.FE,BWA,Botswana,2000,73.9,2000 - 2005
...,...,...,...,...,...,...
527149,SH.DYN.MORT,TZA,Tanzania,2022,40.5,2021 - 2023
527150,SH.DYN.MORT,UGA,Uganda,2022,40.5,2021 - 2023
527151,SH.DYN.MORT,WSM,Samoa,2022,16.1,2021 - 2023
527152,SH.DYN.MORT,ZMB,Zambia,2022,55.6,2021 - 2023


In [15]:
Mortality_counts = WB_Mortality_df['country_name'].value_counts()

print("Counts of entries for each country name:")
print(Mortality_counts)

Counts of entries for each country name:
Madagascar                  69
Malawi                      69
Zambia                      69
Samoa                       69
Uganda                      69
Tanzania                    69
Togo                        69
Chad                        69
Sao Tome and Principe       69
Somalia                     69
Sierra Leone                69
Senegal                     69
Sudan                       69
Rwanda                      69
Nigeria                     69
Mauritania                  69
Tunisia                     69
Mozambique                  69
Lesotho                     69
Liberia                     69
Kenya                       69
Ghana                       69
Gabon                       69
Ethiopia                    69
Egypt, Arab Rep.            69
Djibouti                    69
Central African Republic    69
Botswana                    69
Burkina Faso                69
Angola                      69
Zimbabwe                    6

## Mortality Rate < 5 yr (per 100Th) - Females

In [16]:
WB_Mort_fem_df = WB_africadata_2000_df2.loc[WB_africadata_2000_df2['series_id'] == "SH.DYN.MORT.FE"]
WB_Mort_fem_df = WB_Mort_fem_df.rename(columns = {'value': 'Mortality Rate < 5 yr (per 100Th)'})
WB_Mort_fem_df

,series_id,country_code,country_name,year,Mortality Rate < 5 yr (per 100Th),Year Group
127992,SH.DYN.MORT.FE,MDG,Madagascar,2016,60.1,2016 - 2020
196033,SH.DYN.MORT.FE,TUN,Tunisia,2018,14.1,2016 - 2020
301671,SH.DYN.MORT.FE,AGO,Angola,2000,190.9,2000 - 2005
301672,SH.DYN.MORT.FE,BFA,Burkina Faso,2000,171.5,2000 - 2005
301673,SH.DYN.MORT.FE,BWA,Botswana,2000,73.9,2000 - 2005
...,...,...,...,...,...,...
302571,SH.DYN.MORT.FE,UGA,Uganda,2022,36.1,2021 - 2023
302572,SH.DYN.MORT.FE,WSM,Samoa,2022,14.4,2021 - 2023
302573,SH.DYN.MORT.FE,ZMB,Zambia,2022,50.7,2021 - 2023
302574,SH.DYN.MORT.FE,ZWE,Zimbabwe,2022,43.3,2021 - 2023


In [17]:
# Split up our data into groups based upon 'country_name' and 'year'
WB_Mort_fem_df_groups = WB_Mort_fem_df.groupby(['year', 'country_name'])
WB_Mort_fem_df_groups.head()

,series_id,country_code,country_name,year,Mortality Rate < 5 yr (per 100Th),Year Group
127992,SH.DYN.MORT.FE,MDG,Madagascar,2016,60.1,2016 - 2020
196033,SH.DYN.MORT.FE,TUN,Tunisia,2018,14.1,2016 - 2020
301671,SH.DYN.MORT.FE,AGO,Angola,2000,190.9,2000 - 2005
301672,SH.DYN.MORT.FE,BFA,Burkina Faso,2000,171.5,2000 - 2005
301673,SH.DYN.MORT.FE,BWA,Botswana,2000,73.9,2000 - 2005
...,...,...,...,...,...,...
302571,SH.DYN.MORT.FE,UGA,Uganda,2022,36.1,2021 - 2023
302572,SH.DYN.MORT.FE,WSM,Samoa,2022,14.4,2021 - 2023
302573,SH.DYN.MORT.FE,ZMB,Zambia,2022,50.7,2021 - 2023
302574,SH.DYN.MORT.FE,ZWE,Zimbabwe,2022,43.3,2021 - 2023


In [18]:
avg_Mort_fem_all = WB_Mort_fem_df_groups[['Mortality Rate < 5 yr (per 100Th)']].mean()
avg_Mort_fem_all

Mortality Rate < 5 yr (per 100Th)
year country_name                                               
2000 Angola                                                190.9
     Botswana                                               73.9
     Burkina Faso                                          171.5
     Central African Republic                              157.5
     Chad                                                  175.0
...                                                          ...
2022 Togo                                                   55.3
     Tunisia                                                10.5
     Uganda                                                 36.1
     Zambia                                                 50.7
     Zimbabwe                                               43.3

[713 rows x 1 columns]

In [19]:
avg_Mort_fem_yr = avg_Mort_fem_all.reset_index()

In [20]:
Mort_fem_pivot1 = avg_Mort_fem_yr.pivot(index='country_name', columns='year', values='Mortality Rate < 5 yr (per 100Th)')
Mort_fem_pivot1['Mean Mortality (2000 - 2023)'] = Mort_fem_pivot1.mean(axis=1)
Mort_fem_pivot1

year,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Mean Mortality (2000 - 2023)
country_name,,,,,,,,,,,,,,,,,,,,,
Angola,190.9,185.3,178.7,171.1,162.8,154.2,145.1,136.1,127.3,118.7,...,85.2,81.0,77.1,73.7,70.8,67.9,65.4,63.3,61.0,113.734783
Botswana,73.9,72.9,71.1,70.6,69.7,65.5,61.2,59.5,59.0,56.4,...,46.0,44.3,43.1,40.3,39.9,38.8,37.5,36.1,35.1,53.282609
Burkina Faso,171.5,167.4,162.8,157.8,152.3,146.0,139.6,133.3,127.1,121.3,...,98.8,95.2,91.8,88.5,85.3,82.3,79.3,76.6,73.9,116.830435
Central African Republic,157.5,154.9,152.1,148.8,145.1,141.2,136.7,132.7,128.8,125.0,...,110.3,107.4,104.6,102.9,100.8,98.2,95.4,92.4,89.9,121.378261
Chad,175.0,171.6,168.1,164.6,161.3,157.9,154.6,150.8,147.1,143.2,...,124.5,120.8,117.3,113.6,109.9,106.2,102.9,99.4,96.2,135.617391
Djibouti,92.9,90.9,88.6,86.3,83.9,81.6,79.2,76.7,74.4,72.2,...,61.6,59.5,57.7,55.7,53.8,52.0,50.4,48.8,47.0,68.708696
"Egypt, Arab Rep.",44.8,42.1,39.7,37.6,35.7,34.1,32.5,31.1,29.6,28.3,...,22.7,21.8,20.9,20.2,19.5,18.8,18.1,17.4,16.8,27.517391
Ethiopia,129.8,123.8,117.7,111.4,105.2,99.2,93.3,87.8,82.8,78.0,...,58.8,55.7,52.8,50.2,47.9,45.8,43.8,42.0,40.5,75.543478
Gabon,77.6,75.9,74.1,72.2,70.1,68.0,66.0,63.6,61.3,58.9,...,47.4,45.7,43.9,42.2,40.6,38.9,36.9,35.8,34.7,54.995652


In [21]:
Mort_fem_pivot1['Mean Mortality (2000 - 2023)'] = Mort_fem_pivot1.mean(axis=1)
Mort_fem_pivot1

year,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Mean Mortality (2000 - 2023)
country_name,,,,,,,,,,,,,,,,,,,,,
Angola,190.9,185.3,178.7,171.1,162.8,154.2,145.1,136.1,127.3,118.7,...,85.2,81.0,77.1,73.7,70.8,67.9,65.4,63.3,61.0,113.734783
Botswana,73.9,72.9,71.1,70.6,69.7,65.5,61.2,59.5,59.0,56.4,...,46.0,44.3,43.1,40.3,39.9,38.8,37.5,36.1,35.1,53.282609
Burkina Faso,171.5,167.4,162.8,157.8,152.3,146.0,139.6,133.3,127.1,121.3,...,98.8,95.2,91.8,88.5,85.3,82.3,79.3,76.6,73.9,116.830435
Central African Republic,157.5,154.9,152.1,148.8,145.1,141.2,136.7,132.7,128.8,125.0,...,110.3,107.4,104.6,102.9,100.8,98.2,95.4,92.4,89.9,121.378261
Chad,175.0,171.6,168.1,164.6,161.3,157.9,154.6,150.8,147.1,143.2,...,124.5,120.8,117.3,113.6,109.9,106.2,102.9,99.4,96.2,135.617391
Djibouti,92.9,90.9,88.6,86.3,83.9,81.6,79.2,76.7,74.4,72.2,...,61.6,59.5,57.7,55.7,53.8,52.0,50.4,48.8,47.0,68.708696
"Egypt, Arab Rep.",44.8,42.1,39.7,37.6,35.7,34.1,32.5,31.1,29.6,28.3,...,22.7,21.8,20.9,20.2,19.5,18.8,18.1,17.4,16.8,27.517391
Ethiopia,129.8,123.8,117.7,111.4,105.2,99.2,93.3,87.8,82.8,78.0,...,58.8,55.7,52.8,50.2,47.9,45.8,43.8,42.0,40.5,75.543478
Gabon,77.6,75.9,74.1,72.2,70.1,68.0,66.0,63.6,61.3,58.9,...,47.4,45.7,43.9,42.2,40.6,38.9,36.9,35.8,34.7,54.995652


In [22]:
# Split up our data into groups based upon 'country_name' and 'Year Group'
WB_Mort_fem_groups2 = WB_Mort_fem_df.groupby(['Year Group','country_name'])
WB_Mort_fem_groups2.head()

,series_id,country_code,country_name,year,Mortality Rate < 5 yr (per 100Th),Year Group
127992,SH.DYN.MORT.FE,MDG,Madagascar,2016,60.1,2016 - 2020
196033,SH.DYN.MORT.FE,TUN,Tunisia,2018,14.1,2016 - 2020
301671,SH.DYN.MORT.FE,AGO,Angola,2000,190.9,2000 - 2005
301672,SH.DYN.MORT.FE,BFA,Burkina Faso,2000,171.5,2000 - 2005
301673,SH.DYN.MORT.FE,BWA,Botswana,2000,73.9,2000 - 2005
...,...,...,...,...,...,...
302570,SH.DYN.MORT.FE,TZA,Tanzania,2022,36.6,2021 - 2023
302571,SH.DYN.MORT.FE,UGA,Uganda,2022,36.1,2021 - 2023
302572,SH.DYN.MORT.FE,WSM,Samoa,2022,14.4,2021 - 2023
302573,SH.DYN.MORT.FE,ZMB,Zambia,2022,50.7,2021 - 2023


In [23]:
avg_Mort_fem_country = WB_Mort_fem_groups2[['Mortality Rate < 5 yr (per 100Th)']].mean()
avg_Mort_fem_country

Mortality Rate < 5 yr (per 100Th)
Year Group  country_name                                               
2000 - 2005 Angola                                           173.833333
            Botswana                                          70.616667
            Burkina Faso                                     159.633333
            Central African Republic                         149.933333
            Chad                                             166.416667
...                                                                 ...
2021 - 2023 Togo                                              56.250000
            Tunisia                                           10.850000
            Uganda                                            36.750000
            Zambia                                            51.450000
            Zimbabwe                                          43.900000

[155 rows x 1 columns]

In [24]:
avg_Mort_fem_bin = avg_Mort_fem_country.reset_index()

In [25]:
Mort_fem_pivot2 = avg_Mort_fem_bin.pivot(index='country_name', columns='Year Group', values='Mortality Rate < 5 yr (per 100Th)')
Mort_fem_pivot2['Mean Mortality (2000 - 2023)'] = Mort_fem_pivot2.mean(axis=1)
Mort_fem_pivot2

Year Group,2000 - 2005,2006 - 2010,2011 - 2015,2016 - 2020,2021 - 2023,Mean Mortality (2000 - 2023)
country_name,,,,,,
Angola,173.833333,127.58,91.16,70.98,62.15,105.140667
Botswana,70.616667,58.04,48.16,39.92,35.60,50.467333
Burkina Faso,159.633333,127.46,102.86,85.44,75.25,110.128667
Central African Republic,149.933333,128.90,112.68,100.38,91.15,116.608667
Chad,166.416667,146.98,128.06,109.98,97.80,129.847333
Djibouti,87.366667,74.50,63.64,53.92,47.90,65.465333
"Egypt, Arab Rep.",39.000000,29.70,23.74,19.50,17.10,25.808000
Ethiopia,114.516667,83.10,62.38,48.10,41.25,69.869333
Gabon,72.983333,61.22,49.58,40.50,35.25,51.906667


## Mortality Rate < 5 yr (per 100Th) - Males

In [26]:
WB_Mort_ma_df = WB_africadata_2000_df2.loc[WB_africadata_2000_df2['series_id'] == "SH.DYN.MORT.MA"]
WB_Mort_ma_df

,series_id,country_code,country_name,year,value,Year Group
463915,SH.DYN.MORT.MA,AGO,Angola,2000,214.2,2000 - 2005
463916,SH.DYN.MORT.MA,BFA,Burkina Faso,2000,184.1,2000 - 2005
463917,SH.DYN.MORT.MA,BWA,Botswana,2000,81.4,2000 - 2005
463918,SH.DYN.MORT.MA,CAF,Central African Republic,2000,172.5,2000 - 2005
463919,SH.DYN.MORT.MA,DJI,Djibouti,2000,106.0,2000 - 2005
...,...,...,...,...,...,...
464623,SH.DYN.MORT.MA,TZA,Tanzania,2022,44.2,2021 - 2023
464624,SH.DYN.MORT.MA,UGA,Uganda,2022,44.7,2021 - 2023
464625,SH.DYN.MORT.MA,WSM,Samoa,2022,17.7,2021 - 2023
464626,SH.DYN.MORT.MA,ZMB,Zambia,2022,60.2,2021 - 2023


In [27]:
WB_Mort_ma_df = WB_Mort_fem_df.rename(columns = {'value': 'Mortality Rate < 5 yr (per 100Th)'})
WB_Mort_ma_df

,series_id,country_code,country_name,year,Mortality Rate < 5 yr (per 100Th),Year Group
127992,SH.DYN.MORT.FE,MDG,Madagascar,2016,60.1,2016 - 2020
196033,SH.DYN.MORT.FE,TUN,Tunisia,2018,14.1,2016 - 2020
301671,SH.DYN.MORT.FE,AGO,Angola,2000,190.9,2000 - 2005
301672,SH.DYN.MORT.FE,BFA,Burkina Faso,2000,171.5,2000 - 2005
301673,SH.DYN.MORT.FE,BWA,Botswana,2000,73.9,2000 - 2005
...,...,...,...,...,...,...
302571,SH.DYN.MORT.FE,UGA,Uganda,2022,36.1,2021 - 2023
302572,SH.DYN.MORT.FE,WSM,Samoa,2022,14.4,2021 - 2023
302573,SH.DYN.MORT.FE,ZMB,Zambia,2022,50.7,2021 - 2023
302574,SH.DYN.MORT.FE,ZWE,Zimbabwe,2022,43.3,2021 - 2023


In [28]:
WB_Mort_ma_df_groups = WB_Mort_ma_df.groupby(['year', 'country_name'])
WB_Mort_ma_df_groups.head()

,series_id,country_code,country_name,year,Mortality Rate < 5 yr (per 100Th),Year Group
127992,SH.DYN.MORT.FE,MDG,Madagascar,2016,60.1,2016 - 2020
196033,SH.DYN.MORT.FE,TUN,Tunisia,2018,14.1,2016 - 2020
301671,SH.DYN.MORT.FE,AGO,Angola,2000,190.9,2000 - 2005
301672,SH.DYN.MORT.FE,BFA,Burkina Faso,2000,171.5,2000 - 2005
301673,SH.DYN.MORT.FE,BWA,Botswana,2000,73.9,2000 - 2005
...,...,...,...,...,...,...
302571,SH.DYN.MORT.FE,UGA,Uganda,2022,36.1,2021 - 2023
302572,SH.DYN.MORT.FE,WSM,Samoa,2022,14.4,2021 - 2023
302573,SH.DYN.MORT.FE,ZMB,Zambia,2022,50.7,2021 - 2023
302574,SH.DYN.MORT.FE,ZWE,Zimbabwe,2022,43.3,2021 - 2023


In [29]:
avg_Mort_ma_all = WB_Mort_ma_df_groups[['Mortality Rate < 5 yr (per 100Th)']].mean()
avg_Mort_ma_all

Mortality Rate < 5 yr (per 100Th)
year country_name                                               
2000 Angola                                                190.9
     Botswana                                               73.9
     Burkina Faso                                          171.5
     Central African Republic                              157.5
     Chad                                                  175.0
...                                                          ...
2022 Togo                                                   55.3
     Tunisia                                                10.5
     Uganda                                                 36.1
     Zambia                                                 50.7
     Zimbabwe                                               43.3

[713 rows x 1 columns]

In [30]:
avg_Mort_ma_yr = avg_Mort_ma_all.reset_index()

In [31]:
avg_Mort_ma_pivot1 = avg_Mort_ma_yr.pivot(index='country_name', columns='year', values='Mortality Rate < 5 yr (per 100Th)')
avg_Mort_ma_pivot1

year,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
country_name,,,,,,,,,,,,,,,,,,,,,
Angola,190.9,185.3,178.7,171.1,162.8,154.2,145.1,136.1,127.3,118.7,...,90.3,85.2,81.0,77.1,73.7,70.8,67.9,65.4,63.3,61.0
Botswana,73.9,72.9,71.1,70.6,69.7,65.5,61.2,59.5,59.0,56.4,...,46.8,46.0,44.3,43.1,40.3,39.9,38.8,37.5,36.1,35.1
Burkina Faso,171.5,167.4,162.8,157.8,152.3,146.0,139.6,133.3,127.1,121.3,...,102.5,98.8,95.2,91.8,88.5,85.3,82.3,79.3,76.6,73.9
Central African Republic,157.5,154.9,152.1,148.8,145.1,141.2,136.7,132.7,128.8,125.0,...,112.7,110.3,107.4,104.6,102.9,100.8,98.2,95.4,92.4,89.9
Chad,175.0,171.6,168.1,164.6,161.3,157.9,154.6,150.8,147.1,143.2,...,128.0,124.5,120.8,117.3,113.6,109.9,106.2,102.9,99.4,96.2
Djibouti,92.9,90.9,88.6,86.3,83.9,81.6,79.2,76.7,74.4,72.2,...,63.6,61.6,59.5,57.7,55.7,53.8,52.0,50.4,48.8,47.0
"Egypt, Arab Rep.",44.8,42.1,39.7,37.6,35.7,34.1,32.5,31.1,29.6,28.3,...,23.6,22.7,21.8,20.9,20.2,19.5,18.8,18.1,17.4,16.8
Ethiopia,129.8,123.8,117.7,111.4,105.2,99.2,93.3,87.8,82.8,78.0,...,62.2,58.8,55.7,52.8,50.2,47.9,45.8,43.8,42.0,40.5
Gabon,77.6,75.9,74.1,72.2,70.1,68.0,66.0,63.6,61.3,58.9,...,49.3,47.4,45.7,43.9,42.2,40.6,38.9,36.9,35.8,34.7


In [32]:
avg_Mort_ma_pivot1['Mean Mortality (2000 - 2023)'] = avg_Mort_ma_pivot1.mean(axis=1)
avg_Mort_ma_pivot1

year,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Mean Mortality (2000 - 2023)
country_name,,,,,,,,,,,,,,,,,,,,,
Angola,190.9,185.3,178.7,171.1,162.8,154.2,145.1,136.1,127.3,118.7,...,85.2,81.0,77.1,73.7,70.8,67.9,65.4,63.3,61.0,113.734783
Botswana,73.9,72.9,71.1,70.6,69.7,65.5,61.2,59.5,59.0,56.4,...,46.0,44.3,43.1,40.3,39.9,38.8,37.5,36.1,35.1,53.282609
Burkina Faso,171.5,167.4,162.8,157.8,152.3,146.0,139.6,133.3,127.1,121.3,...,98.8,95.2,91.8,88.5,85.3,82.3,79.3,76.6,73.9,116.830435
Central African Republic,157.5,154.9,152.1,148.8,145.1,141.2,136.7,132.7,128.8,125.0,...,110.3,107.4,104.6,102.9,100.8,98.2,95.4,92.4,89.9,121.378261
Chad,175.0,171.6,168.1,164.6,161.3,157.9,154.6,150.8,147.1,143.2,...,124.5,120.8,117.3,113.6,109.9,106.2,102.9,99.4,96.2,135.617391
Djibouti,92.9,90.9,88.6,86.3,83.9,81.6,79.2,76.7,74.4,72.2,...,61.6,59.5,57.7,55.7,53.8,52.0,50.4,48.8,47.0,68.708696
"Egypt, Arab Rep.",44.8,42.1,39.7,37.6,35.7,34.1,32.5,31.1,29.6,28.3,...,22.7,21.8,20.9,20.2,19.5,18.8,18.1,17.4,16.8,27.517391
Ethiopia,129.8,123.8,117.7,111.4,105.2,99.2,93.3,87.8,82.8,78.0,...,58.8,55.7,52.8,50.2,47.9,45.8,43.8,42.0,40.5,75.543478
Gabon,77.6,75.9,74.1,72.2,70.1,68.0,66.0,63.6,61.3,58.9,...,47.4,45.7,43.9,42.2,40.6,38.9,36.9,35.8,34.7,54.995652


In [33]:
# Split up our data into groups based upon 'country_name' and 'Year Group'
WB_Mort_ma_groups2 = WB_Mort_ma_df.groupby(['Year Group','country_name'])
WB_Mort_ma_groups2.head()

,series_id,country_code,country_name,year,Mortality Rate < 5 yr (per 100Th),Year Group
127992,SH.DYN.MORT.FE,MDG,Madagascar,2016,60.1,2016 - 2020
196033,SH.DYN.MORT.FE,TUN,Tunisia,2018,14.1,2016 - 2020
301671,SH.DYN.MORT.FE,AGO,Angola,2000,190.9,2000 - 2005
301672,SH.DYN.MORT.FE,BFA,Burkina Faso,2000,171.5,2000 - 2005
301673,SH.DYN.MORT.FE,BWA,Botswana,2000,73.9,2000 - 2005
...,...,...,...,...,...,...
302570,SH.DYN.MORT.FE,TZA,Tanzania,2022,36.6,2021 - 2023
302571,SH.DYN.MORT.FE,UGA,Uganda,2022,36.1,2021 - 2023
302572,SH.DYN.MORT.FE,WSM,Samoa,2022,14.4,2021 - 2023
302573,SH.DYN.MORT.FE,ZMB,Zambia,2022,50.7,2021 - 2023


In [34]:
avg_Mort_ma_country = WB_Mort_ma_groups2[['Mortality Rate < 5 yr (per 100Th)']].mean()
avg_Mort_ma_country

Mortality Rate < 5 yr (per 100Th)
Year Group  country_name                                               
2000 - 2005 Angola                                           173.833333
            Botswana                                          70.616667
            Burkina Faso                                     159.633333
            Central African Republic                         149.933333
            Chad                                             166.416667
...                                                                 ...
2021 - 2023 Togo                                              56.250000
            Tunisia                                           10.850000
            Uganda                                            36.750000
            Zambia                                            51.450000
            Zimbabwe                                          43.900000

[155 rows x 1 columns]

In [35]:
avg_Mort_ma_bin = avg_Mort_ma_country.reset_index()

In [36]:
Mort_ma_pivot2 = avg_Mort_ma_bin.pivot(index='country_name', columns='Year Group', values='Mortality Rate < 5 yr (per 100Th)')
Mort_ma_pivot2

Year Group,2000 - 2005,2006 - 2010,2011 - 2015,2016 - 2020,2021 - 2023
country_name,,,,,
Angola,173.833333,127.58,91.16,70.98,62.15
Botswana,70.616667,58.04,48.16,39.92,35.60
Burkina Faso,159.633333,127.46,102.86,85.44,75.25
Central African Republic,149.933333,128.90,112.68,100.38,91.15
Chad,166.416667,146.98,128.06,109.98,97.80
Djibouti,87.366667,74.50,63.64,53.92,47.90
"Egypt, Arab Rep.",39.000000,29.70,23.74,19.50,17.10
Ethiopia,114.516667,83.10,62.38,48.10,41.25
Gabon,72.983333,61.22,49.58,40.50,35.25


In [37]:
Mort_ma_pivot2['Mean Mortality (2000 - 2023)'] = Mort_ma_pivot2.mean(axis=1)
Mort_ma_pivot2

Year Group,2000 - 2005,2006 - 2010,2011 - 2015,2016 - 2020,2021 - 2023,Mean Mortality (2000 - 2023)
country_name,,,,,,
Angola,173.833333,127.58,91.16,70.98,62.15,105.140667
Botswana,70.616667,58.04,48.16,39.92,35.60,50.467333
Burkina Faso,159.633333,127.46,102.86,85.44,75.25,110.128667
Central African Republic,149.933333,128.90,112.68,100.38,91.15,116.608667
Chad,166.416667,146.98,128.06,109.98,97.80,129.847333
Djibouti,87.366667,74.50,63.64,53.92,47.90,65.465333
"Egypt, Arab Rep.",39.000000,29.70,23.74,19.50,17.10,25.808000
Ethiopia,114.516667,83.10,62.38,48.10,41.25,69.869333
Gabon,72.983333,61.22,49.58,40.50,35.25,51.906667


## Mortality Rate < 5 yr (per 100Th) - Combined

In [38]:
WB_Mort_combo_df = WB_africadata_2000_df2.loc[WB_africadata_2000_df2['series_id'] == "SH.DYN.MORT"]
WB_Mort_combo_df

,series_id,country_code,country_name,year,value,Year Group
526439,SH.DYN.MORT,AGO,Angola,2000,202.9,2000 - 2005
526440,SH.DYN.MORT,BFA,Burkina Faso,2000,178.0,2000 - 2005
526441,SH.DYN.MORT,BWA,Botswana,2000,77.8,2000 - 2005
526442,SH.DYN.MORT,CAF,Central African Republic,2000,165.2,2000 - 2005
526443,SH.DYN.MORT,DJI,Djibouti,2000,99.7,2000 - 2005
...,...,...,...,...,...,...
527149,SH.DYN.MORT,TZA,Tanzania,2022,40.5,2021 - 2023
527150,SH.DYN.MORT,UGA,Uganda,2022,40.5,2021 - 2023
527151,SH.DYN.MORT,WSM,Samoa,2022,16.1,2021 - 2023
527152,SH.DYN.MORT,ZMB,Zambia,2022,55.6,2021 - 2023


In [39]:
WB_Mort_combo_df = WB_Mort_fem_df.rename(columns = {'value': 'Mortality Rate < 5 yr (per 100Th)'})
WB_Mort_combo_df

,series_id,country_code,country_name,year,Mortality Rate < 5 yr (per 100Th),Year Group
127992,SH.DYN.MORT.FE,MDG,Madagascar,2016,60.1,2016 - 2020
196033,SH.DYN.MORT.FE,TUN,Tunisia,2018,14.1,2016 - 2020
301671,SH.DYN.MORT.FE,AGO,Angola,2000,190.9,2000 - 2005
301672,SH.DYN.MORT.FE,BFA,Burkina Faso,2000,171.5,2000 - 2005
301673,SH.DYN.MORT.FE,BWA,Botswana,2000,73.9,2000 - 2005
...,...,...,...,...,...,...
302571,SH.DYN.MORT.FE,UGA,Uganda,2022,36.1,2021 - 2023
302572,SH.DYN.MORT.FE,WSM,Samoa,2022,14.4,2021 - 2023
302573,SH.DYN.MORT.FE,ZMB,Zambia,2022,50.7,2021 - 2023
302574,SH.DYN.MORT.FE,ZWE,Zimbabwe,2022,43.3,2021 - 2023


In [40]:
# Split up our data into groups based upon 'country_name' and 'Year Group'
WB_Mort_combo_groups = WB_Mort_combo_df.groupby(['year', 'country_name'])
WB_Mort_combo_groups.head()

,series_id,country_code,country_name,year,Mortality Rate < 5 yr (per 100Th),Year Group
127992,SH.DYN.MORT.FE,MDG,Madagascar,2016,60.1,2016 - 2020
196033,SH.DYN.MORT.FE,TUN,Tunisia,2018,14.1,2016 - 2020
301671,SH.DYN.MORT.FE,AGO,Angola,2000,190.9,2000 - 2005
301672,SH.DYN.MORT.FE,BFA,Burkina Faso,2000,171.5,2000 - 2005
301673,SH.DYN.MORT.FE,BWA,Botswana,2000,73.9,2000 - 2005
...,...,...,...,...,...,...
302571,SH.DYN.MORT.FE,UGA,Uganda,2022,36.1,2021 - 2023
302572,SH.DYN.MORT.FE,WSM,Samoa,2022,14.4,2021 - 2023
302573,SH.DYN.MORT.FE,ZMB,Zambia,2022,50.7,2021 - 2023
302574,SH.DYN.MORT.FE,ZWE,Zimbabwe,2022,43.3,2021 - 2023


In [41]:
avg_Mort_combo_all = WB_Mort_combo_groups[['Mortality Rate < 5 yr (per 100Th)']].mean()
avg_Mort_combo_all

Mortality Rate < 5 yr (per 100Th)
year country_name                                               
2000 Angola                                                190.9
     Botswana                                               73.9
     Burkina Faso                                          171.5
     Central African Republic                              157.5
     Chad                                                  175.0
...                                                          ...
2022 Togo                                                   55.3
     Tunisia                                                10.5
     Uganda                                                 36.1
     Zambia                                                 50.7
     Zimbabwe                                               43.3

[713 rows x 1 columns]

In [42]:
avg_Mort_combo_yr = avg_Mort_combo_all.reset_index()

In [43]:
Mort_combo_pivot1 = avg_Mort_combo_yr.pivot(index='country_name', columns='year', values='Mortality Rate < 5 yr (per 100Th)')
Mort_combo_pivot1

year,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
country_name,,,,,,,,,,,,,,,,,,,,,
Angola,190.9,185.3,178.7,171.1,162.8,154.2,145.1,136.1,127.3,118.7,...,90.3,85.2,81.0,77.1,73.7,70.8,67.9,65.4,63.3,61.0
Botswana,73.9,72.9,71.1,70.6,69.7,65.5,61.2,59.5,59.0,56.4,...,46.8,46.0,44.3,43.1,40.3,39.9,38.8,37.5,36.1,35.1
Burkina Faso,171.5,167.4,162.8,157.8,152.3,146.0,139.6,133.3,127.1,121.3,...,102.5,98.8,95.2,91.8,88.5,85.3,82.3,79.3,76.6,73.9
Central African Republic,157.5,154.9,152.1,148.8,145.1,141.2,136.7,132.7,128.8,125.0,...,112.7,110.3,107.4,104.6,102.9,100.8,98.2,95.4,92.4,89.9
Chad,175.0,171.6,168.1,164.6,161.3,157.9,154.6,150.8,147.1,143.2,...,128.0,124.5,120.8,117.3,113.6,109.9,106.2,102.9,99.4,96.2
Djibouti,92.9,90.9,88.6,86.3,83.9,81.6,79.2,76.7,74.4,72.2,...,63.6,61.6,59.5,57.7,55.7,53.8,52.0,50.4,48.8,47.0
"Egypt, Arab Rep.",44.8,42.1,39.7,37.6,35.7,34.1,32.5,31.1,29.6,28.3,...,23.6,22.7,21.8,20.9,20.2,19.5,18.8,18.1,17.4,16.8
Ethiopia,129.8,123.8,117.7,111.4,105.2,99.2,93.3,87.8,82.8,78.0,...,62.2,58.8,55.7,52.8,50.2,47.9,45.8,43.8,42.0,40.5
Gabon,77.6,75.9,74.1,72.2,70.1,68.0,66.0,63.6,61.3,58.9,...,49.3,47.4,45.7,43.9,42.2,40.6,38.9,36.9,35.8,34.7


In [44]:
# Split up our data into groups based upon 'country_name' and 'Year Group'
WB_Mort_combo_groups2 = WB_Mort_combo_df.groupby(['Year Group','country_name'])
WB_Mort_combo_groups2.head()

,series_id,country_code,country_name,year,Mortality Rate < 5 yr (per 100Th),Year Group
127992,SH.DYN.MORT.FE,MDG,Madagascar,2016,60.1,2016 - 2020
196033,SH.DYN.MORT.FE,TUN,Tunisia,2018,14.1,2016 - 2020
301671,SH.DYN.MORT.FE,AGO,Angola,2000,190.9,2000 - 2005
301672,SH.DYN.MORT.FE,BFA,Burkina Faso,2000,171.5,2000 - 2005
301673,SH.DYN.MORT.FE,BWA,Botswana,2000,73.9,2000 - 2005
...,...,...,...,...,...,...
302570,SH.DYN.MORT.FE,TZA,Tanzania,2022,36.6,2021 - 2023
302571,SH.DYN.MORT.FE,UGA,Uganda,2022,36.1,2021 - 2023
302572,SH.DYN.MORT.FE,WSM,Samoa,2022,14.4,2021 - 2023
302573,SH.DYN.MORT.FE,ZMB,Zambia,2022,50.7,2021 - 2023


In [45]:
avg_Mort_combo_country = WB_Mort_combo_groups2[['Mortality Rate < 5 yr (per 100Th)']].mean()
avg_Mort_combo_country

Mortality Rate < 5 yr (per 100Th)
Year Group  country_name                                               
2000 - 2005 Angola                                           173.833333
            Botswana                                          70.616667
            Burkina Faso                                     159.633333
            Central African Republic                         149.933333
            Chad                                             166.416667
...                                                                 ...
2021 - 2023 Togo                                              56.250000
            Tunisia                                           10.850000
            Uganda                                            36.750000
            Zambia                                            51.450000
            Zimbabwe                                          43.900000

[155 rows x 1 columns]

In [46]:
avg_Mort_combo_bin = avg_Mort_combo_country.reset_index()

In [47]:
Mort_combo_pivot2 = avg_Mort_combo_bin.pivot(index='country_name', columns='Year Group', values='Mortality Rate < 5 yr (per 100Th)')
Mort_combo_pivot2

Year Group,2000 - 2005,2006 - 2010,2011 - 2015,2016 - 2020,2021 - 2023
country_name,,,,,
Angola,173.833333,127.58,91.16,70.98,62.15
Botswana,70.616667,58.04,48.16,39.92,35.60
Burkina Faso,159.633333,127.46,102.86,85.44,75.25
Central African Republic,149.933333,128.90,112.68,100.38,91.15
Chad,166.416667,146.98,128.06,109.98,97.80
Djibouti,87.366667,74.50,63.64,53.92,47.90
"Egypt, Arab Rep.",39.000000,29.70,23.74,19.50,17.10
Ethiopia,114.516667,83.10,62.38,48.10,41.25
Gabon,72.983333,61.22,49.58,40.50,35.25


## Africa Female Mortality Rate

In [48]:
Mort_fem_pivot3 = avg_Mort_fem_bin.pivot(index='Year Group', columns='country_name', values='Mortality Rate < 5 yr (per 100Th)')
Mort_fem_pivot3

country_name,Angola,Botswana,Burkina Faso,Central African Republic,Chad,Djibouti,"Egypt, Arab Rep.",Ethiopia,Gabon,Ghana,...,Senegal,Sierra Leone,Somalia,Sudan,Tanzania,Togo,Tunisia,Uganda,Zambia,Zimbabwe
Year Group,,,,,,,,,,,,,,,,,,,,,
2000 - 2005,173.833333,70.616667,159.633333,149.933333,166.416667,87.366667,39.00,114.516667,72.983333,83.316667,...,104.433333,201.50,164.483333,87.95,104.266667,103.15,22.766667,116.683333,123.05,85.25
2006 - 2010,127.580000,58.040000,127.460000,128.900000,146.980000,74.500000,29.70,83.100000,61.220000,68.440000,...,69.560000,167.04,157.460000,72.88,72.340000,87.84,17.940000,79.860000,81.56,85.10
2011 - 2015,91.160000,48.160000,102.860000,112.680000,128.060000,63.640000,23.74,62.380000,49.580000,54.640000,...,51.780000,136.76,134.800000,62.68,54.860000,74.96,16.560000,56.940000,66.02,62.62
2016 - 2020,70.980000,39.920000,85.440000,100.380000,109.980000,53.920000,19.50,48.100000,40.500000,43.920000,...,39.540000,109.24,114.160000,53.68,43.780000,63.44,13.960000,42.920000,57.56,49.26
2021 - 2023,62.150000,35.600000,75.250000,91.150000,97.800000,47.900000,17.10,41.250000,35.250000,38.450000,...,33.700000,95.60,101.500000,47.55,37.300000,56.25,10.850000,36.750000,51.45,43.90


In [49]:
Mort_fem_pivot3['Mean Mortality (2000 - 2023)'] = Mort_fem_pivot3.mean(axis=1)
Mort_fem_pivot3

country_name,Angola,Botswana,Burkina Faso,Central African Republic,Chad,Djibouti,"Egypt, Arab Rep.",Ethiopia,Gabon,Ghana,...,Sierra Leone,Somalia,Sudan,Tanzania,Togo,Tunisia,Uganda,Zambia,Zimbabwe,Mean Mortality (2000 - 2023)
Year Group,,,,,,,,,,,,,,,,,,,,,
2000 - 2005,173.833333,70.616667,159.633333,149.933333,166.416667,87.366667,39.00,114.516667,72.983333,83.316667,...,201.50,164.483333,87.95,104.266667,103.15,22.766667,116.683333,123.05,85.25,109.147312
2006 - 2010,127.580000,58.040000,127.460000,128.900000,146.980000,74.500000,29.70,83.100000,61.220000,68.440000,...,167.04,157.460000,72.88,72.340000,87.84,17.940000,79.860000,81.56,85.10,85.025161
2011 - 2015,91.160000,48.160000,102.860000,112.680000,128.060000,63.640000,23.74,62.380000,49.580000,54.640000,...,136.76,134.800000,62.68,54.860000,74.96,16.560000,56.940000,66.02,62.62,68.690968
2016 - 2020,70.980000,39.920000,85.440000,100.380000,109.980000,53.920000,19.50,48.100000,40.500000,43.920000,...,109.24,114.160000,53.68,43.780000,63.44,13.960000,42.920000,57.56,49.26,57.582581
2021 - 2023,62.150000,35.600000,75.250000,91.150000,97.800000,47.900000,17.10,41.250000,35.250000,38.450000,...,95.60,101.500000,47.55,37.300000,56.25,10.850000,36.750000,51.45,43.90,51.275806


## Africa Male Mortality Rate

In [50]:
Mort_ma_pivot3 = avg_Mort_ma_bin.pivot(index='Year Group', columns='country_name', values='Mortality Rate < 5 yr (per 100Th)')
Mort_ma_pivot3['Mean Mortality (2000 - 2023)'] = Mort_ma_pivot3.mean(axis=1)
Mort_ma_pivot3

country_name,Angola,Botswana,Burkina Faso,Central African Republic,Chad,Djibouti,"Egypt, Arab Rep.",Ethiopia,Gabon,Ghana,...,Sierra Leone,Somalia,Sudan,Tanzania,Togo,Tunisia,Uganda,Zambia,Zimbabwe,Mean Mortality (2000 - 2023)
Year Group,,,,,,,,,,,,,,,,,,,,,
2000 - 2005,173.833333,70.616667,159.633333,149.933333,166.416667,87.366667,39.00,114.516667,72.983333,83.316667,...,201.50,164.483333,87.95,104.266667,103.15,22.766667,116.683333,123.05,85.25,109.147312
2006 - 2010,127.580000,58.040000,127.460000,128.900000,146.980000,74.500000,29.70,83.100000,61.220000,68.440000,...,167.04,157.460000,72.88,72.340000,87.84,17.940000,79.860000,81.56,85.10,85.025161
2011 - 2015,91.160000,48.160000,102.860000,112.680000,128.060000,63.640000,23.74,62.380000,49.580000,54.640000,...,136.76,134.800000,62.68,54.860000,74.96,16.560000,56.940000,66.02,62.62,68.690968
2016 - 2020,70.980000,39.920000,85.440000,100.380000,109.980000,53.920000,19.50,48.100000,40.500000,43.920000,...,109.24,114.160000,53.68,43.780000,63.44,13.960000,42.920000,57.56,49.26,57.582581
2021 - 2023,62.150000,35.600000,75.250000,91.150000,97.800000,47.900000,17.10,41.250000,35.250000,38.450000,...,95.60,101.500000,47.55,37.300000,56.25,10.850000,36.750000,51.45,43.90,51.275806


## Africa Combined Mortality Rate

In [51]:
Mort_combo_pivot3 = avg_Mort_fem_bin.pivot(index='Year Group', columns='country_name', values='Mortality Rate < 5 yr (per 100Th)')
Mort_combo_pivot3['Mean Mortality (2000 - 2023)'] = Mort_combo_pivot3.mean(axis=1)
Mort_combo_pivot3

country_name,Angola,Botswana,Burkina Faso,Central African Republic,Chad,Djibouti,"Egypt, Arab Rep.",Ethiopia,Gabon,Ghana,...,Sierra Leone,Somalia,Sudan,Tanzania,Togo,Tunisia,Uganda,Zambia,Zimbabwe,Mean Mortality (2000 - 2023)
Year Group,,,,,,,,,,,,,,,,,,,,,
2000 - 2005,173.833333,70.616667,159.633333,149.933333,166.416667,87.366667,39.00,114.516667,72.983333,83.316667,...,201.50,164.483333,87.95,104.266667,103.15,22.766667,116.683333,123.05,85.25,109.147312
2006 - 2010,127.580000,58.040000,127.460000,128.900000,146.980000,74.500000,29.70,83.100000,61.220000,68.440000,...,167.04,157.460000,72.88,72.340000,87.84,17.940000,79.860000,81.56,85.10,85.025161
2011 - 2015,91.160000,48.160000,102.860000,112.680000,128.060000,63.640000,23.74,62.380000,49.580000,54.640000,...,136.76,134.800000,62.68,54.860000,74.96,16.560000,56.940000,66.02,62.62,68.690968
2016 - 2020,70.980000,39.920000,85.440000,100.380000,109.980000,53.920000,19.50,48.100000,40.500000,43.920000,...,109.24,114.160000,53.68,43.780000,63.44,13.960000,42.920000,57.56,49.26,57.582581
2021 - 2023,62.150000,35.600000,75.250000,91.150000,97.800000,47.900000,17.10,41.250000,35.250000,38.450000,...,95.60,101.500000,47.55,37.300000,56.25,10.850000,36.750000,51.45,43.90,51.275806
